In [1]:
import numpy as np
import pandas as pd
import string
import re


In [129]:
all_reviews=pd.read_csv('all_critic_opinions_file.csv')
all_reviews=all_reviews.drop('Unnamed: 0',axis=1)
all_reviews=all_reviews.reset_index(drop=True)
all_reviews.head()

,CRating,TMeter,Title,Review,Critic
0,3/5,87%,Ant-Man and the Wasp (2018),"Ant-Man and the Wasp is an airy, nimble piece ...",alex-abad-santos
1,3.5/5,67%,Ocean's 8 (2018),"Ocean's 8, at its most endearing, is a slick, ...",alex-abad-santos
2,4.5/5,83%,Deadpool 2 (2018),"A delirious exhibition for the bawdy, acerbic ...",alex-abad-santos
3,NaN,83%,Avengers: Infinity War (2018),"For the first time in a while, I can't wait to...",alex-abad-santos
4,4.5/5,97%,Black Panther (2018),Coogler and his talented cast and team tell a ...,alex-abad-santos


In [80]:
all_reviews.shape

(670325, 5)

In [81]:
print('Number of films: '+str(len(all_reviews['Title'].unique())),\
      'Number of critics: '+str(len(all_reviews['Critic'].unique())))

Number of films: 38850 Number of critics: 3260


__Data cleaning:__

In [130]:
all_reviews =all_reviews[all_reviews['Title'].notnull()]
all_reviews_films=all_reviews.drop(all_reviews[all_reviews['Title'].str.contains('TV')].index)
all_reviews_films=all_reviews_films.drop(all_reviews_films[all_reviews_films['Review']=='Review'].index)
all_reviews_films['Year']=all_reviews_films.Title.str[-5:-1]
all_reviews_films=all_reviews_films[['CRating','TMeter','Title','Year','Review','Critic']]
all_reviews_films['Title']=all_reviews_films.Title.str[:-7]
all_reviews_films['TMeter']=all_reviews_films.TMeter.str[:-1]
all_reviews_films.shape

(555341, 6)

In [131]:
all_reviews_films=all_reviews_films.drop(all_reviews_films[all_reviews_films['TMeter']=='No Score Ye'].index)
all_reviews_films.shape

(531960, 6)

__Changing Tomatometer (film rating) values to numeric and replacing all values >= 0.6 with 1 (Tomatometer:Fresh) and all values <0.6 with 0 (Tomatometer:Rotten) in additional column TMeter1:__

In [132]:
all_reviews_films['TMeter']=pd.to_numeric(all_reviews_films['TMeter'])/100
all_reviews_films['TMeter1']=all_reviews_films['TMeter']
all_reviews_films.loc[all_reviews_films['TMeter1'] >=0.6, 'TMeter1'] = 1
all_reviews_films.loc[all_reviews_films['TMeter1'] <0.6, 'TMeter1'] = 0

In [91]:
all_reviews_films.sample(7)

,CRating,TMeter,Title,Year,Review,Critic,TMeter1
115589,NaN,0.21,Ma ma,2016,Even knowing that Ma Ma is intended to be a wo...,sherilyn-connelly,0.0
429834,4.5/5,0.94,Christmas in July,1940,"Tragically underseen, this Preston Sturges com...",christopher-null,1.0
365761,4/5,0.80,Unsane,2018,Unsane is sometimes plodding for a movie that ...,ian-mantgani,1.0
163907,4/5,0.96,Restrepo,2010,First-rate stuff.‐ Sky Cinema - EDIT Read Mo...,tim-evans,1.0
429907,3/5,0.88,The Glenn Miller Story,1953,Quote not available.‐ Filmcritic.com - EDIT ...,christopher-null,1.0
185777,8/10,0.86,The Omen,1976,All references to prophecy and the Antichrist ...,bill-gibron,1.0
379517,A-,0.97,Tangerine,2015,Tangerine is touching for its non-condescendin...,joe-mcgovern,1.0


__Cleaning the Review column:__

In [133]:
all_reviews_films=all_reviews_films.drop(all_reviews_films[all_reviews_films['Review'].str.contains('Quote not available')].index)
all_reviews_films=all_reviews_films.drop(all_reviews_films[all_reviews_films['Review'].str.contains('Full review in')].index)
all_reviews_films=all_reviews_films.drop(all_reviews_films[all_reviews_films['Review'].str.contains('Full Review in')].index)

In [104]:
all_reviews_films.shape

(511571, 7)

In [105]:
print('Number of films: '+str(len(all_reviews_films['Title'].unique())),\
      'Number of critics: '+str(len(all_reviews_films['Critic'].unique())))

Number of films: 21401 Number of critics: 2525


__Removing all the films that have less then 5 reviews:__

In [134]:
counts=all_reviews_films['Title'].value_counts()
all_reviews_films=all_reviews_films[all_reviews_films['Title'].isin(counts[counts>5].index)]

In [107]:
print('Number of critic reviews: '+str(all_reviews_films.shape[0]), 'Number of films: '+str(len(all_reviews_films['Title'].value_counts())))

Number of critic reviews: 490078 Number of films: 13630


__Review text cleaning to remove unnecessary information about the review source:__

In [137]:
all_reviews_films['Review'].iloc[1]

"Ocean's 8, at its most endearing, is a slick, glamorous romp that makes you yearn for three more hours with its impossibly charismatic crew.‐ Vox  - EDIT  Read More | Posted Jun 5, 2018"

In [138]:
all_reviews_films['Review'].iloc[1]

"Ocean's 8, at its most endearing, is a slick, glamorous romp that makes you yearn for three more hours with its impossibly charismatic crew.‐ Vox  - EDIT  Read More | Posted Jun 5, 2018"

In [139]:
all_reviews_films['Review']=all_reviews_films['Review'].str.split('.').str[:-1]
all_reviews_films['Review'].loc[1]

["Ocean's 8, at its most endearing, is a slick, glamorous romp that makes you yearn for three more hours with its impossibly charismatic crew"]

In [140]:
all_reviews_films['Review']=all_reviews_films['Review'].str.join('')
all_reviews_films['Review'].iloc[1]

"Ocean's 8, at its most endearing, is a slick, glamorous romp that makes you yearn for three more hours with its impossibly charismatic crew"

__Removing special characters, numbers, as well as words like 'film' and 'movie' from the text:__

In [141]:
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('\d','')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('\'','')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('/',' ')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('-',' ')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('`','')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('film','')
all_reviews_films['Review']=all_reviews_films['Review'].str.replace('movie','')
all_reviews_films['Review'].iloc[1]

'Oceans , at its most endearing, is a slick, glamorous romp that makes you yearn for three more hours with its impossibly charismatic crew'

In [143]:
all_reviews_films=all_reviews_films[all_reviews_films['Review'].isnull()==False]

In [144]:
all_reviews_films.to_csv('all_critic_opinions_movies.csv')

In [148]:
all_reviews_films1=pd.read_csv('all_critic_opinions_movies.csv')
all_reviews_films1=all_reviews_films1.drop('Unnamed: 0',axis=1)
all_reviews_films1=all_reviews_films1.reset_index(drop=True)
all_reviews_films1=all_reviews_films1.drop(['CRating','Critic'],axis=1)
all_reviews_films1.sample(7)

,TMeter,Title,Year,Review,TMeter1
373932,0.92,Network,1976,Chayefsky was apparently serious about much of...,1.0
423014,0.90,Calvary,2014,Its a that works almost like a sermon: It los...,1.0
177292,0.19,Assassin's Creed,2016,"""Assassins Creed"" doesnt allow us to hold the ...",0.0
359813,0.29,The Hunted,2003,"Ludicrous, plotless, ho hum tale of lurid conf...",0.0
79889,0.85,2046,2005,is a must for fans of Wong Kar Wai with its s...,1.0
396702,0.96,The Innocents,1961,"So much is suggested, and so little shown",1.0
150060,0.69,The Runaways,2010,Music video director Floria Sigismondis The Ru...,1.0


In [149]:
all_reviews_films1=all_reviews_films1[all_reviews_films1['Review'].isnull()==False]


__Grouping all the reviews by the title and year of the movie:__

In [150]:
all_reviews_films2=all_reviews_films1.groupby(['Title','Year'])['Review'].apply(' '.join).reset_index()
all_reviews_films2.shape

(14328, 3)

In [151]:
all_reviews_films2[all_reviews_films2['Title']=='The Revenant']

,Title,Year,Review
12309,The Revenant,2012,What starts as funny and engaging winds up plo...
12310,The Revenant,2015,All the more incredible for not needing D glas...


In [152]:
all_reviews_films3=all_reviews_films1.groupby(['Title','Year'])['TMeter','TMeter1'].max().reset_index()
all_reviews_films3.shape

(14328, 4)

In [153]:
all_reviews_films3[(all_reviews_films3['Title']=='The Revenant')|(all_reviews_films3['Title']=='Tully')]

,Title,Year,TMeter,TMeter1
12309,The Revenant,2012,0.56,0.0
12310,The Revenant,2015,0.80,1.0
13309,Tully,2002,0.81,1.0
13310,Tully,2018,0.87,1.0


In [154]:
all_reviews_films4=pd.merge(all_reviews_films2,all_reviews_films3,on=('Title','Year'))
all_reviews_films4.shape

(14328, 5)

In [155]:
all_reviews_films4.sample(5)

,Title,Year,Review,TMeter,TMeter1
11498,The Hunger Games: Mockingjay - Part 2,2015,"Hate to say it, but ""The Hunger Games"" franchi...",0.70,1.0
8583,Pride and Glory,2008,"Intelligently directed, beautifully shot (by D...",0.35,0.0
11143,The Film Critic,2015,gets away with all the ic tropes & clichés tha...,0.45,0.0
2433,Chocolate,2008,JeeJa Yanin is a sensational discovery but thi...,0.74,1.0
6579,"Liza, Liza, Skies Are Grey",2017,"Smitten with his characters, Sanders takes the...",0.22,0.0


__Example of the grouped critic's review for one film:__

In [156]:
all_reviews_films4[all_reviews_films4['Title']=='A Beautiful Mind']

,Title,Year,Review,TMeter,TMeter1
230,A Beautiful Mind,2001,The kind of that makes you wish theyd just do...,0.75,1.0


In [157]:
all_reviews_films4['Review'].loc[230]

'The kind of  that makes you wish theyd just do away with the Oscars altogether, A Beautiful Mind is as tasteful as a dentists office, and about as exciting Humanistic and engaging piece of popular entertainment As strong a performance as Russell Crowe has ever delivered A brilliant performance by Russell Crowe, who takes his audience on a terrifying journey inside a man tormented by self created mental demons, propels A Beautiful Mind The second, idealised, sentimentalised half of the  is torture, as we plod through the routine Im here to help psychiatrist, the walls plastered with cut up newspapers and the what happens when he stops taking the medicine stuff In a good cast, Jennifer Connelly earns your complete admiration Harrowing, heartbreaking and deeply human A beautiful  At its most effective when it seems to lose the plot in a scrambled second act that posits the Cold War as a collective paranoid delusion, the  reverts to type (and to fact) for a sentimental anti climax All the

__Grouped reviews contain the titles of the films which are not useful for the text analysis and have to be removed:__

In [158]:
np.warnings.filterwarnings('ignore')

In [159]:
for i in range(all_reviews_films4.shape[0]):
    title=all_reviews_films4['Title'].loc[i]
    all_reviews_films4['Review'].loc[i]=all_reviews_films4['Review'].loc[i].replace(title,'')
print(all_reviews_films4['Review'].loc[230])

The kind of  that makes you wish theyd just do away with the Oscars altogether,  is as tasteful as a dentists office, and about as exciting Humanistic and engaging piece of popular entertainment As strong a performance as Russell Crowe has ever delivered A brilliant performance by Russell Crowe, who takes his audience on a terrifying journey inside a man tormented by self created mental demons, propels  The second, idealised, sentimentalised half of the  is torture, as we plod through the routine Im here to help psychiatrist, the walls plastered with cut up newspapers and the what happens when he stops taking the medicine stuff In a good cast, Jennifer Connelly earns your complete admiration Harrowing, heartbreaking and deeply human A beautiful  At its most effective when it seems to lose the plot in a scrambled second act that posits the Cold War as a collective paranoid delusion, the  reverts to type (and to fact) for a sentimental anti climax All the pieces are there and Ron Howard 

In [160]:
all_reviews_films4.to_csv('all_reviews_films4.csv')